In [1]:
import os
import cv2
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.datasets import mnist
from keras.utils import *
import matplotlib.pyplot as plt
from keras import optimizers
from keras import losses
import numpy as np
import random


Using TensorFlow backend.


In [2]:
def get_fscore(cm):
    precision = cm[1][0]/(cm[0][0]+cm[1][0])
    recall =  cm[1][0]/(cm[1][0]+cm[1][1])

    f1 = 2*precision*recall/(precision+recall)
    print("Melanoma F1 score:",f1)
   
    precision = cm[0][0]/(cm[0][0]+cm[0][1])
    recall =  cm[0][0]/(cm[0][0]+cm[1][0])

    f1 = 2*precision*recall/(precision+recall)
    print("Non-melanoma F1 score:",f1)

In [3]:
batch_size = 1280
gt_all = os.listdir('gt')
    
batch_train_x = np.zeros((batch_size,64,64,3))
batch_train_y = np.zeros((batch_size,1))
for a in range(batch_size):
    i = random.randint(0,len(gt_all)-1)

    name = gt_all[i]
    name = name.replace('_segmentation','')
    name = name.replace('png','jpg')
    if name in os.listdir('melanoma'):
        img = cv2.imread(os.path.join('melanoma',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_train_x[a] = img
            batch_train_y[a] = 1
    else:
        img = cv2.imread(os.path.join('others',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_train_x[a] = img
            batch_train_y[a] = 0
batch_train_x = batch_train_x.astype('float32')
batch_train_y = batch_train_y.astype('float32')
batch_train_y = to_categorical(batch_train_y,2)
print('Done')

Done


In [4]:
batch_size = 320
gt_all = os.listdir('gt')
    
batch_val_x = np.zeros((batch_size,64,64,3))
batch_val_y = np.zeros((batch_size,1))


for a in range(batch_size):
    i = random.randint(0,len(gt_all)-1)
 
    name = gt_all[i]
    name = name.replace('_segmentation','')
    name = name.replace('png','jpg')
    if name in os.listdir('melanoma'):
        img = cv2.imread(os.path.join('melanoma',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_val_x[a] = img
            batch_val_y[a] = 1
    else:
        img = cv2.imread(os.path.join('others',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_val_x[a] = img
            batch_val_y[a] = 0
batch_val_x = batch_val_x.astype('float32')
batch_val_y = batch_val_y.astype('float32')
batch_val_y = to_categorical(batch_val_y,2)
print('Done')

Done


In [5]:
batch_size = 200
gt_all = os.listdir('gt')
    
batch_test_x = np.zeros((batch_size,64,64,3))
batch_test_y = np.zeros((batch_size,1))

for a in range(batch_size):
    i = random.randint(0,len(gt_all)-1)
 
    name = gt_all[i]
    name = name.replace('_segmentation','')
    name = name.replace('png','jpg')
    if name in os.listdir('melanoma'):
        img = cv2.imread(os.path.join('melanoma',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_test_x[a] = img
            batch_test_y[a] = 1
    else:
        img = cv2.imread(os.path.join('others',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_test_x[a] = img
            batch_test_y[a] = 0
batch_test_x = batch_test_x.astype('float32')
batch_test_y = batch_test_y.astype('float32')
batch_test_y = to_categorical(batch_test_y,2)
print('Done')

Done


In [8]:
class_weight = {0.6,0.2}

In [11]:
model1 = Sequential()
model1.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape = (64,64,3)))
model1.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model1.add(Conv2D(64, (5, 5), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(1000, activation='relu'))
model1.add(Dense(2, activation='softmax'))
model1.compile(loss=losses.categorical_crossentropy,
              optimizer=optimizers.SGD(lr=0.5),
              metrics=['accuracy'])
model1.fit(batch_train_x,batch_train_y,validation_data = (batch_val_x,batch_val_y),epochs=20, batch_size=200,class_weight = class_weight, verbose=1)

Train on 1280 samples, validate on 320 samples
Epoch 1/20
1280/1280 [==============================] - 49s 38ms/step - loss: 4.4183 - acc: 0.7258 - val_loss: 2.0651 - val_acc: 0.8719
Epoch 2/20
1280/1280 [==============================] - 48s 37ms/step - loss: 2.9592 - acc: 0.8164 - val_loss: 2.0651 - val_acc: 0.8719
Epoch 3/20
1280/1280 [==============================] - 48s 38ms/step - loss: 2.9592 - acc: 0.8164 - val_loss: 2.0651 - val_acc: 0.8719
Epoch 4/20
1280/1280 [==============================] - 48s 38ms/step - loss: 2.9592 - acc: 0.8164 - val_loss: 2.0651 - val_acc: 0.8719
Epoch 5/20
1280/1280 [==============================] - 49s 38ms/step - loss: 2.9592 - acc: 0.8164 - val_loss: 2.0651 - val_acc: 0.8719
Epoch 6/20
1280/1280 [==============================] - 48s 38ms/step - loss: 2.9592 - acc: 0.8164 - val_loss: 2.0651 - val_acc: 0.8719
Epoch 7/20
1280/1280 [==============================] - 48s 37ms/step - loss: 2.9592 - acc: 0.8164 - val_loss: 2.0651 - val_acc: 0.8719
E

In [12]:
score = model1.evaluate(batch_test_x,batch_test_y,verbose=1)

200/200 [==============================] - 3s 16ms/step


In [13]:
print('Test loss:', score[0])
print('Test accuracy:', score[1]*100,'%')

Test loss: 2.82066669941
Test accuracy: 82.5 %


In [14]:
y_pred = model1.predict(batch_test_x)
y_test = batch_test_y
cm = confusion_matrix(y_test.argmax(axis=1),y_pred.argmax(axis=1))
get_fscore(cm)

Melanoma F1 score: 0.297872340426
Non-melanoma F1 score: 0.904109589041
